In [1]:
def get_date(delta=0):
    """
    По умолчанию возвращает текущую дату в виде строки формата '%Y-%m-%d'. 
    Чтобы получить любую прошедшую дату, нужно в качестве аргумента передать
    количество дней от сегодняшней даты. Например 1 это вчера.
    """
    from datetime import datetime, timedelta
    
    date_format = '%Y-%m-%d'
    today = datetime.now()   
    yesterday = str((today - timedelta(days=delta)).strftime(date_format)) # вчерашняя дата в виде строки
    return yesterday

In [66]:
def get_soup(url):      
    '''
    Функция возвращает объект класса BS
    ''' 
    from requests.exceptions import HTTPError
    from bs4 import BeautifulSoup as BS
    import requests 
    
    try:
        response = requests.get(url)
        # если ответ успешен, исключения задействованы не будут
        response.raise_for_status()
    except HTTPError as http_err:
        print(f'HTTP error occurred: {http_err}')
    except Exception as err:
        print(f'Other error occurred: {err}')
    else:
        pass
        # print('Success!')
    soup = BS(response.text, features="html.parser")     
    return soup

In [63]:
def get_data_in_liveinternet(date):
    """
    Парсит с сайта www.liveinternet.ru количество просмотров и посетителей сайта Глотовский СДК за определенную дату.
    Дата передается в качестве аргумента в виде строки формата '%Y-%m-%d'
    """
    import pandas as pd
    import numpy as np       
    import re
    from bs4 import BeautifulSoup as BS    
    
    url = f'https://www.liveinternet.ru/stat/glotovo-club.ru/index.html?date={date}' # ссылка на счетчик    

    # парсим посещаемость сайта Глотовского СДК
    soup = get_soup(url)   
    title = soup.findAll('title')[0].renderContents().decode('UTF-8')
    wiev = soup.findAll('tr', {'align':'right'})[1].findAll('td')[2].text.split()[0] # число просмотров
    users = soup.findAll('tr', {'align':'right'})[1].findAll('td')[14].text.split()[0] # кличество посетителей
    s = {'дата': date, 'просмотры': int(wiev), 'посетители': int(users)}    
    return s

In [64]:
def write_to_googlesheet(row, table_name, path_to_credential): 
    """
    добавляет строку в гугл таблицу.
    row - словарь, значения которого будут добавлены в гугл таблицу
    table_name - имя гугл таблицы в которую будет добавлена запись
    path_to_credential - имя файла ключа с расширением json
    """
    # чтобы все работало нужно произвести настройки на стороне гугла по ссылке https://console.developers.google.com/ 

    import gspread
    from oauth2client.service_account import ServiceAccountCredentials
    import pandas as pd   

    scope = ['https://spreadsheets.google.com/feeds', # эти ссылки не трогаем
             'https://www.googleapis.com/auth/drive']
    credentials = ServiceAccountCredentials.from_json_keyfile_name(path_to_credential, scope)
    client = gspread.authorize(credentials)  # запускаем клиент для связи с таблицами
    work_sheet = client.open(table_name)

    # выбираем первый лист в таблице
    sheet1 = work_sheet.sheet1  
    s = (row['дата'], row['просмотры'], row['посетители'])
    sheet1.append_row(s)

In [65]:
l = get_data_in_liveinternet(get_date())
write_to_googlesheet(l, 'log', 'Club dashboard 79755-2f6e609fb3c4.json')